## LSTM Encoder Decoder method for Motion prediction

Objective of the competition is to predict the future trajectories of other vehicles using the past information (Bird's eye view of the scene containing agents detected by perception system, past trajectories lane information, traffic lights etc).


Since this is sequential problem, I thought of using LSTM based models. Basic idea is as follows:

[LSTM_baseline idea](https://www.kaggle.com/suryajrrafl/lstm-baseline-weights?select=lstm+baseline+idea.jpg)

This is my first attempt at kaggle competition, pytorch and LSTM models. Suggestions are most welcome.


**REFERENCES**

Some helper functions in this notebook were taken from the great public kernels avaiable. 

[Great reference notebook using Resnet model](https://www.kaggle.com/huanvo/lyft-complete-train-and-prediction-pipeline)

[Pytorch baseline train](https://www.kaggle.com/pestipeti/pytorch-baseline-train)

## Import libraries

In [1]:
# torch imports
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
import torch.functional as F

# l5kit imports
import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory

# common imports
import os
import random
import time
import pandas as pd
from typing import Dict
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from prettytable import PrettyTable
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [2]:
l5kit.__version__

'1.1.0'

In [3]:
torch.cuda.is_available()

True

## Helper functions

### Random seed generation function

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

### Number of trainable parameters in model

In [7]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(total_trainable_params)
    return total_trainable_params

### Loss function

In [4]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Configs - constants for model

In [11]:
PREDICTION_LENGTH        = 50
NUMBER_OF_HISTORY_FRAMES = 10
RASTER_IMG_SIZE          = 224
PIXEL_SIZE               = 0.5
NUM_MODES                = 1
NUMBER_OF_FUTURE_FRAMES = 50

### TRAIN FROM WHERE LEFT OFF, CHANGE THE STARTING INDICES VARIABLE ACCORDINGLY
TRAIN_START_INDICES = 0

In [12]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': "lyft-motion-prediction-autonomous-vehicles/",
    'model_params': {
        'model_architecture': 'resnet18',
        'history_num_frames': NUMBER_OF_HISTORY_FRAMES,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': PREDICTION_LENGTH,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "LSTM_EncDec_R18",
        'weight_path': "",
        'lr': 1e-3,
        'train': True,
        'predict': False
    },

    'raster_params': {
        'raster_size': [RASTER_IMG_SIZE, RASTER_IMG_SIZE],
        'pixel_size': [PIXEL_SIZE, PIXEL_SIZE],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'sample_data_loader': {
        'key': 'scenes/sample.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },

    'train_params': {
        'train_start_index' : 0,
        'max_num_steps': 25,
        'checkpoint_every_n_steps': 5,
    }
}

## LSTM Encoder Decoder Model 

In [6]:
class LSTM_EncDec(nn.Module):

    def __init__(self, cfg: Dict, output_size, embed_size, use_other_features= False):
        super(LSTM_EncDec, self).__init__()

        # Resnet architecture as such for most part of encoding
        architecture = cfg["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True)
        self.backbone = backbone

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        # This is 512 for resnet18 and resnet34; And it is 2048 for the other resnets
        if architecture == "resnet50":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512
            
        # X, Y coords for the future positions (output shape: batch_sizex50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        #num_targets = 2 * self.future_len
        #self.num_targets = num_targets
        
        # including other feautres as part of encoded hidden state
        other_agent_features = num_history_channels + 6
        
        # LSTM layers
        #self.output_size = output_size
        #self.embed_size = embed_size
        
        if use_other_features ==True:
            self.hidden_size = backbone_out_features + other_agent_features
        else:
            self.hidden_size = backbone_out_features
        
        self.linear_in  = nn.Linear(output_size, embed_size)
        self.decoder    = nn.LSTMCell(embed_size, self.hidden_size, bias=True)
        self.linear_out = nn.Linear(self.hidden_size, output_size) 
            
    def Resnet_forward(self, x):
        # Resnet encoding layers
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        return x
    
    
    def forward(self, x, hidden):
        
        # input size to embedded size
        embedded = F.relu(self.linear_in(x))
        
        # passsing through LSTM layer
        LSTM_out = self.decoder(embedded, hidden)
        
        # converting from hidden state size to output size
        prediction = self.linear_out(LSTM_out[0])
        
        return prediction, LSTM_out

In [9]:
def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    # extracting required array elements and moving to device
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)    
    history_positions = data['history_positions'].to(device).view(TRAIN_BATCH_SIZE, -1)
    centroid = data['centroid'].to(device).float()
    yaw = data['yaw'].to(device).view(TRAIN_BATCH_SIZE, 1).float()
    extent = data['extent'].to(device)
    
    batch_size = inputs.shape[0]    
    
    # concatenating vehicle data
    agent_data = torch.cat((history_positions, centroid, yaw, extent), dim=1)
    
    # concatenating image data and vehicle data to get encoded state
    image_data = model.Resnet_forward(inputs)
    decoder_hidden = torch.cat((image_data, agent_data), dim=1)
    
    # preallocating memory
    prediction = torch.zeros_like(targets).to(device)
    loss = 0.0
    
    # change to last history position?
    decoder_input = torch.zeros(batch_size, 1, 2).to(device) 

    # Decode hidden state in future trajectory
    for t in range(PREDICTION_LENGTH):
        decoder_output, decoder_hidden = model(decoder_input, decoder_hidden)
        prediction[:, t, :] = decoder_output

        # Update loss
        loss += criterion(decoder_output[:, :2], targets[:, di, :], target_availabilities[:])

        # Use own predictions as inputs at next step
        decoder_input = decoder_output
    
    # Get average loss for pred_len
    loss = loss / PREDICTION_LENGTH 
        
    return loss, prediction

## Load the training data

In [16]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
rasterizer = build_rasterizer(cfg, dm)

In [17]:
# ===== INIT TRAIN DATASET============================================================
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

In [18]:
print('Length of Train dataset is ' ,len(train_dataset))
print("==================================TRAIN DATA==================================")
print(train_dataset)

Length of Train dataset is  22496709
==================================TRAIN DATA==================================
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [19]:
len(train_dataset)

22496709

In [31]:
sampled_indices = np.random.choice(len(train_dataset), size = len(train_dataset), replace = False)
print('Before slicing, start indices are ', sampled_indices[0:10])

Before slicing, start indices are  [ 4544818 11783688  2821987 10532781  1993967 15711361  7726097   547516
 17556178  7275034]


In [32]:
TRAIN_START_INDICES

0

In [33]:
sampled_indices = sampled_indices[TRAIN_START_INDICES:]
print('After slicing, start indices are ', sampled_indices[0:10])

After slicing, start indices are  [ 4544818 11783688  2821987 10532781  1993967 15711361  7726097   547516
 17556178  7275034]


In [34]:
Datasampler = SubsetRandomSampler(sampled_indices)

In [35]:
train_dataloader = DataLoader(train_dataset, sampler=Datasampler, batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])

## CUDA device && encoding model

In [36]:
# ==== INIT MODEL=================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device {device}')

device cuda:0


In [43]:
# decide on hyperparameters
embed_size = 8
#n_layers   = 2

# instantiate an RNN
model = LSTM_EncDec(cfg, embed_size, output_size, use_other_features=True)
model.to(device);

In [49]:
model.hidden_size

615

In [50]:
find_no_of_trainable_params(model.backbone.conv1)

78400

In [52]:
a = resnet18(pretrained=True)
find_no_of_trainable_params(a)

11689512

In [48]:
Total_model_params = find_no_of_trainable_params(model)
Backbone_params = find_no_of_trainable_params(model.backbone)
print(f'There are  {Backbone_params} parameters in Backbone of LSTM model')
print(f'There are  {Total_model_params - Backbone_params} parameters in rest of LSTM model')

There are  11758504 parameters in Backbone of LSTM model
There are  1530155 parameters in rest of LSTM model


In [45]:
## loading the pretrained weights
#model.load_state_dict(torch.load(cfg['model_params']['weight_path']))

In [42]:
## Adam optimiser function
optimizer = optim.Adam(model.parameters(), lr=cfg["model_params"]["lr"])

## Training

In [13]:
TRAIN_START_INDICES

0

In [14]:
print('TRAINING ABOUT TO START ... FROM ', TRAIN_START_INDICES, '  BATCH')

TRAINING ABOUT TO START ... FROM  0   BATCH


In [ ]:
# ==== TRAINING LOOP =========================================================
if cfg["model_params"]["train"]:
    
    tr_it = iter(train_dataloader)
    progress_bar = tqdm(range(TRAIN_START_INDICES, 
                              TRAIN_START_INDICES + cfg["train_params"]["max_num_steps"]))
    num_iter = cfg["train_params"]["max_num_steps"]
    losses_train = []
    iterations = []
    metrics = []
    times = []
    model_name = cfg["model_params"]["model_name"]
    start = time.time()
    hidden_state = None
    
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
            
        # Forward pass
        model.train()
        torch.set_grad_enabled(True)
        loss, hidden_state, _, _ = forward(data, model, hidden_state, device)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses_train.append(loss.item())

        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")
        if i % cfg['train_params']['checkpoint_every_n_steps'] == 0:
            sample_number = i * cfg['train_data_loader']['batch_size']            
            state = {
              'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict(),
            }
            torch.save(state, f'{model_name}_{sample_number}k.pth')
            iterations.append(i)
            metrics.append(np.mean(losses_train))
            times.append((time.time()-start)/60)

    sample_number = i * cfg['train_data_loader']['batch_size']
    results = pd.DataFrame({'iterations': iterations, 'metrics (avg)': metrics, 'elapsed_time (mins)': times})
    results.to_csv(f"train_metrics_{model_name}_{sample_number}k.csv", index = False)
    train_losses_csv = pd.DataFrame({'iteration': TRAIN_START_INDICES + np.arange(len(losses_train)), 
                                 'losses_train': losses_train})
    train_losses_csv.to_csv(f"train_losses_{model_name}_{sample_number}k.csv", index = False)
    print(f"Total training time is {(time.time()-start)/60} mins")
    print(results.head())